In [1]:
!pip install rouge_score
!pip install textstat
!pip install datasets
import os
import re
# from transformers import AutoTokenizer, AutoModelForQuestionAnswering

from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
import textstat
import pandas as pd

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=030d62a4042c3531b30de0143d6864d9e778fa34ea7426bee9967ed92146031d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fs

In [ ]:
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict

# Disable WandB integration
os.environ["WANDB_DISABLED"] = "true"

# Use Legal BERT
model_name = "nlpaueb/legal-bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Load and validate the dataset
csv_file = '/corrected_tos.csv'
df = pd.read_csv(csv_file)
df = df.dropna(subset=['Sentence', 'Label'])
label_mapping = {"High Risk": 0, "Low Risk": 1, "Positive Point": 2}
df['Label'] = df['Label'].map(label_mapping)

# Use a subset of the data for debugging
train_test_split = 0.8
train_df = df.sample(frac=train_test_split, random_state=42)
test_df = df.drop(train_df.index)
train_df = train_df.sample(frac=0.1, random_state=42)  # Use 10% for training (adjust as needed)
test_df = test_df.sample(frac=0.1, random_state=42)  # Use 10% for testing (adjust as needed)

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
dataset = DatasetDict({"train": train_dataset, "test": test_dataset})

# Tokenize the dataset
def tokenize_function(example):
    tokenized = tokenizer(example['Sentence'], truncation=True, padding="max_length", max_length=256)
    tokenized["labels"] = example["Label"]  # Rename Label column to labels
    return tokenized

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['Sentence', 'Label', '__index_level_0__'])
tokenized_datasets = tokenized_datasets.with_format("torch")

# Define training arguments with reduced batch size and mixed precision
training_args = TrainingArguments(
    output_dir="./legal_bert_classification",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,  # Reduce batch size
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    gradient_accumulation_steps=8,  # Simulate larger batch size
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    fp16=True,  # Enable mixed precision
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/282 [00:00<?, ? examples/s]

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-10-e5ad6100ff39>:62: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
0,No log,0.646136
1,0.916800,0.401648
2,0.419700,0.329425


TrainOutput(global_step=24, training_loss=0.5991721153259277, metrics={'train_runtime': 68.0787, 'train_samples_per_second': 12.427, 'train_steps_per_second': 0.353, 'total_flos': 101035552407552.0, 'train_loss': 0.5991721153259277, 'epoch': 2.845070422535211})

In [14]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from textstat import textstat
import numpy as np

# Get model predictions on the test dataset
predictions = trainer.predict(tokenized_datasets["test"])

# Convert logits to predicted labels
predicted_labels = np.argmax(predictions.predictions, axis=1)

# Map numerical labels back to textual labels
reverse_label_mapping = {0: "High Risk", 1: "Low Risk", 2: "Positive Point"}
predicted_texts = [reverse_label_mapping[label] for label in predicted_labels]

# Convert actual_labels to a list of integers
actual_labels = [int(label) for label in tokenized_datasets["test"]["labels"]]

# Map numerical labels back to textual labels
actual_texts = [reverse_label_mapping[label] for label in actual_labels]

# Function to evaluate BLEU, ROUGE, and Accuracy
def evaluate_metrics(reference_texts, predicted_texts):
    # Initialize BLEU scorer with smoothing
    smoothing = SmoothingFunction().method1
    bleu_scores = []
    rouge = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge_scores = {"rouge1": [], "rougeL": []}

    # Iterate through reference and predicted texts
    for ref, pred in zip(reference_texts, predicted_texts):
        # Calculate BLEU score with unigram overlap and smoothing
        bleu = sentence_bleu(
            [ref.split()], pred.split(), weights=(1, 0, 0, 0), smoothing_function=smoothing
        )
        bleu_scores.append(bleu)

        # Calculate ROUGE scores
        rouge_score = rouge.score(ref, pred)
        rouge_scores["rouge1"].append(rouge_score["rouge1"].fmeasure)
        rouge_scores["rougeL"].append(rouge_score["rougeL"].fmeasure)

    # Calculate average BLEU and ROUGE scores
    avg_bleu = np.mean(bleu_scores)
    avg_rouge = {metric: np.mean(scores) for metric, scores in rouge_scores.items()}

    return avg_bleu, avg_rouge

# Evaluate BLEU and ROUGE scores
avg_bleu, avg_rouge = evaluate_metrics(actual_texts, predicted_texts)

# Evaluate accuracy
accuracy = sum(1 for a, p in zip(actual_labels, predicted_labels) if a == p) / len(actual_labels)

# Function to evaluate readability
def evaluate_readability(predicted_texts):
    # Filter texts with at least 2 words
    filtered_texts = [text for text in predicted_texts if len(text.split()) > 1]
    if not filtered_texts:  # Avoid errors if no valid texts
        return {"readability": 0, "avg_sentence_length": 0, "avg_syllables_per_word": 0}

    # Calculate Flesch Reading Ease
    readability_scores = [textstat.flesch_reading_ease(text) for text in filtered_texts]
    avg_readability = np.mean(readability_scores)

    # Calculate Average Sentence Length
    avg_sentence_length = np.mean([len(text.split()) for text in filtered_texts])

    # Calculate Average Syllables Per Word
    avg_syllables = np.mean([
        textstat.syllable_count(text) / len(text.split())
        for text in filtered_texts if len(text.split()) > 0
    ])

    return {
        "readability": avg_readability,
        "avg_sentence_length": avg_sentence_length,
        "avg_syllables_per_word": avg_syllables
    }
# Evaluate readability score
readability_metrics = evaluate_readability(predicted_texts)

# Print the evaluation results
print(f"Accuracy: {accuracy:.4f}")
print(f"Average BLEU Score: {avg_bleu:.4f}")
print("Average ROUGE Scores:")
for metric, score in avg_rouge.items():
    print(f"  {metric.upper()}: {score:.4f}")
print(f"Average Readability Score: {readability_metrics['readability']:.4f}")
print(f"Average Sentence Length: {readability_metrics['avg_sentence_length']:.4f} words")
print(f"Average Syllables Per Word: {readability_metrics['avg_syllables_per_word']:.4f}")

Accuracy: 0.9286
Average BLEU Score: 0.9500
Average ROUGE Scores:
  ROUGE1: 0.9500
  ROUGEL: 0.9500
Average Readability Score: 120.2100
Average Sentence Length: 2.0000 words
Average Syllables Per Word: 1.0000
